In [22]:
import awswrangler as wr
import datetime
from datetime import date 

today = date.today()
day, month, year = (today - datetime.timedelta(days=60)).strftime("%d/%m/%Y").split('/')
period = year + month
period

'202102'

In [4]:
q = """
CREATE TABLE coe_analytics_tmp.ao_cross3_vehjoin1
WITH (
  format='PARQUET',
  external_location='s3://rimac-analytics-temporal/individuals/Antonio Ordonez/data/base_cross/ao_cross3_vehjoin1'
) AS
select a.* ,
b.grupo_analytics_concat,
b.prima as prima_tot_miscli ,
b.combo_riesgos as combo_riesgos_miscli, 
b.combo_canales as combo_canales_miscli,
b.first_periodo
from coe_analytics_tmp.ao_cross3_veh_target a 
left join coe_analytics_tmp.ao_cross3_tmp04 b 
on a.cuc = b.cuc and a.periodo = b.periodo;
"""

wr.s3.delete_objects('s3://rimac-analytics-temporal/individuals/Antonio Ordonez/data/base_cross/ao_cross3_vehjoin1')

query_id = wr.athena.start_query_execution('DROP TABLE IF EXISTS coe_analytics_tmp.ao_cross3_vehjoin1;', database='coe_analytics_tmp')
res = wr.athena.wait_query(query_execution_id=query_id)

query_id = wr.athena.start_query_execution(q, database='coe_analytics_tmp', )
res = wr.athena.wait_query(query_execution_id=query_id)

In [5]:
q = """
CREATE TABLE coe_analytics_tmp.ao_cross3_vehjoin2
with (
 format = 'parquet',
 parquet_compression = 'snappy',
 external_location = 's3://rimac-analytics-temporal/individuals/Antonio Ordonez/data/base_cross/ao_cross3_vehjoin2'
)  as 

with t as (
select *,
 row_number() over (partition by cuc, periodo order by  beneficio_pps desc) num_row
from  coe_analytics_tmp.gy_cross_data_salud_titular
  where cuc is not null
  )
select A.*, B.cod_afiliado, B.n_atenciones, B.beneficio_pps, B.n_atenciones_maternidad, B.valor_venta_procedimiento,
B.flag_paciente_cronico, B.gasto_maternidad, B.avg_dias_dev_reembolso, B.n_tipos_beneficio, B.gastos_excluidos_r,
 B.valor_vta_proc_r, B.pct_gastos_excluidos_r, B.n_preexistencias 
FROM coe_analytics_tmp.ao_cross3_vehjoin1 A
LEFT JOIN t B 
on A.cuc = B.cuc and A.periodo = B.periodo and B.num_row = 1 ;
"""

wr.s3.delete_objects('s3://rimac-analytics-temporal/individuals/Antonio Ordonez/data/base_cross/ao_cross3_vehjoin2')

query_id = wr.athena.start_query_execution('DROP TABLE IF EXISTS coe_analytics_tmp.ao_cross3_vehjoin2;', database='coe_analytics_tmp')
res = wr.athena.wait_query(query_execution_id=query_id)

query_id = wr.athena.start_query_execution(q, database='coe_analytics_tmp', )
res = wr.athena.wait_query(query_execution_id=query_id)

In [6]:
q = """
CREATE TABLE coe_analytics_tmp.ao_cross3_vehjoin3
with (
 format = 'parquet',
 parquet_compression = 'snappy',
 external_location = 's3://rimac-analytics-temporal/individuals/Antonio Ordonez/data/base_cross/ao_cross3_vehjoin3'
)  as 

with t as (
select *,
 row_number() over (partition by cuc, periodo order by  beneficio_pps desc) num_row
from  coe_analytics_tmp.gy_cross_data_salud_dependientes
  where cuc is not null
  )
select A.*, B.cod_afiliado_vinculacion, B.n_dependientes, B.n_atenciones as n_atenciones_dep, B.beneficio_pps as beneficio_pps_dep, 
B.n_atenciones_maternidad as n_atenciones_maternidad_dep, B.valor_venta_procedimiento as valor_vta_proc_dep, B.flag_paciente_cronico as flag_paciente_cronico_dep,
B.gasto_maternidad as gasto_maternidad_dep, B.avg_dias_dev_reembolso as avg_dias_dev_r_dep, B.n_tipos_beneficio as n_tipos_beneficio_dep, B.gastos_excluidos_r as gastos_excluidos_r_dep,
B.valor_vta_proc_r as valor_vta_proc_r_dep,  B.gastos_excluidos_r / B.valor_vta_proc_r as pct_gastos_excluidos_r_dep
FROM coe_analytics_tmp.ao_cross3_vehjoin2 A
LEFT JOIN t B 
on A.cuc = B.cuc and A.periodo = B.periodo and B.num_row = 1 ;
"""

wr.s3.delete_objects('s3://rimac-analytics-temporal/individuals/Antonio Ordonez/data/base_cross/ao_cross3_vehjoin3')

query_id = wr.athena.start_query_execution('DROP TABLE IF EXISTS coe_analytics_tmp.ao_cross3_vehjoin3;', database='coe_analytics_tmp')
res = wr.athena.wait_query(query_execution_id=query_id)

query_id = wr.athena.start_query_execution(q, database='coe_analytics_tmp', )
res = wr.athena.wait_query(query_execution_id=query_id)

In [7]:
q = """
CREATE TABLE coe_analytics_tmp.ao_cross3_vehjoin4
with (
 format = 'parquet',
 parquet_compression = 'snappy',
 external_location = 's3://rimac-analytics-temporal/individuals/Antonio Ordonez/data/base_cross/ao_cross3_vehjoin4'
)  as 

SELECT A.*, B.scoreingreso, B.rango, B.alta, B.situacion, B.e_giro, B.tipotrabajador 
FROM coe_analytics_tmp.ao_cross3_vehjoin3 A
LEFT JOIN coe_analytics_tmp.ao_crossde_dep B
ON A.cuc = B.cuc;
"""

wr.s3.delete_objects('s3://rimac-analytics-temporal/individuals/Antonio Ordonez/data/base_cross/ao_cross3_vehjoin4')

query_id = wr.athena.start_query_execution('DROP TABLE IF EXISTS coe_analytics_tmp.ao_cross3_vehjoin4;', database='coe_analytics_tmp')
res = wr.athena.wait_query(query_execution_id=query_id)

query_id = wr.athena.start_query_execution(q, database='coe_analytics_tmp', )
res = wr.athena.wait_query(query_execution_id=query_id)

In [8]:
q = """
CREATE TABLE coe_analytics_tmp.ao_cross3_vehjoin5
WITH (
  format='PARQUET',
  external_location='s3://rimac-analytics-temporal/individuals/Antonio Ordonez/data/base_cross/ao_cross3_vehjoin5'
) AS
select *,
    row_number() over (partition by cuc, periodo_target order by periodo desc) montn_desc,
    row_number() over (partition by cuc, periodo_target order by periodo ) montn_asc
from coe_analytics_tmp.ao_cross3_vehjoin4;
"""

wr.s3.delete_objects('s3://rimac-analytics-temporal/individuals/Antonio Ordonez/data/base_cross/ao_cross3_vehjoin5')

query_id = wr.athena.start_query_execution('DROP TABLE IF EXISTS coe_analytics_tmp.ao_cross3_vehjoin5;', database='coe_analytics_tmp')
res = wr.athena.wait_query(query_execution_id=query_id)

query_id = wr.athena.start_query_execution(q, database='coe_analytics_tmp', )
res = wr.athena.wait_query(query_execution_id=query_id)

In [12]:
q = """
CREATE TABLE coe_analytics_tmp.ao_cross3_vehjoin6
WITH (
  format='PARQUET',
  external_location='s3://rimac-analytics-temporal/individuals/Antonio Ordonez/data/base_cross/ao_cross3_vehjoin6'
) AS
select cuc, periodo_target, veh,
-- SALUD
MAX(n_atenciones) max_atenciones_mes,
sum(n_atenciones) tot_atenciones_mes,
sum(beneficio_pps) tot_beneficio_pps,
avg(beneficio_pps) avg_beneficio_pps,
max(case when montn_desc = 1 then flag_paciente_cronico  else null end) flag_paciente_cronico,   

MAX(n_atenciones_dep) max_atencionesdep_mes,
sum(n_atenciones_dep) tot_atencionesdep_mes,
sum(beneficio_pps_dep) tot_beneficio_pps_dep,
avg(beneficio_pps_dep) avg_beneficio_pps_dep,
max(case when montn_desc = 1 then flag_paciente_cronico_dep  else null end) flag_paciente_cronico_dep,    
max(case when montn_desc = 1 then n_atenciones_maternidad_dep  else null end) n_atenciones_maternidad_dep,       

-- DATA EXTERNA
max(case when montn_desc = 1 then scoreingreso   else null end) scoreingreso,
max(case when montn_desc = 1 then situacion   else null end) situacion  ,
max(case when montn_desc = 1 then tipotrabajador else null end) tipotrabajador,
max(case when montn_desc = 1 then e_giro    else null end) e_giro,

-- MIS CLIENTES
max(case when montn_desc = 1 then grupo_analytics_concat   else null end) grupo_analytics_concat   ,
max(case when montn_desc = 1 then combo_riesgos_miscli   else null end) combo_riesgos_miscli   ,
max(case when montn_desc = 1 then combo_canales_miscli   else null end) combo_canales_miscli    ,
max(case when montn_desc = 1 then first_periodo   else null end) first_periodo,
max(case when montn_asc = 1 then grupo_analytics_concat   else null end) first_grupo_analytics_concat   ,
max(case when montn_desc = 1 then prima_tot_miscli   else null end) last_prima_tot_miscli, 
avg(prima_tot_miscli ) avg_prima_tot_miscli ,
count(distinct periodo) num_meses_miscli

from coe_analytics_tmp.ao_cross3_vehjoin5
group by cuc, periodo_target, veh;
"""

wr.s3.delete_objects('s3://rimac-analytics-temporal/individuals/Antonio Ordonez/data/base_cross/ao_cross3_vehjoin6')

query_id = wr.athena.start_query_execution('DROP TABLE IF EXISTS coe_analytics_tmp.ao_cross3_vehjoin6;', database='coe_analytics_tmp')
res = wr.athena.wait_query(query_execution_id=query_id)

query_id = wr.athena.start_query_execution(q, database='coe_analytics_tmp', )
res = wr.athena.wait_query(query_execution_id=query_id)

In [4]:
q = """
CREATE TABLE coe_analytics_tmp.ao_cross3_vehjoin7
WITH (
  format='PARQUET',
  external_location='s3://rimac-analytics-temporal/individuals/Antonio Ordonez/data/base_cross/ao_cross3_vehjoin7'
) AS
select A.*, B.dem_sexo, B.dem_edad, B.dem_estado_civil, B.dem_grado_instruccion, B.dem_ocupacion, B.dem_lima_agrup, B.dem_dep_agrup,
 B.dem_cono_agrup, B.dem_nse_rimac, B.dem_nse_rimac_6, B.rcc_flag_rcc, B.rcc_cal_gral, B.rcc_num_tc_sbs, B.rcc_num_vehic_sbs, B.rcc_num_hipot_sbs, 
 B.rcc_saldo_tc_sbs, B.rcc_saldo_veh_sbs, B.rcc_saldo_hip_sbs, B.rcc_cal_gral_6, B.rcc_num_tc_sbs_6, B.rcc_num_vehic_sbs_6, B.rcc_num_hipot_sbs_6,
  B.rcc_saldo_tc_sbs_6, B.rcc_saldo_veh_sbs_6, B.rcc_saldo_hip_sbs_6, rcc_linea_tcmax, rcc_linea_tcmax_6
FROM ads_mt.mt_persona B
right JOIN coe_analytics_tmp.ao_cross3_vehjoin6 A ON A.cuc = B.cuc and A.periodo_target = B.periodo;

"""

wr.s3.delete_objects('s3://rimac-analytics-temporal/individuals/Antonio Ordonez/data/base_cross/ao_cross3_vehjoin7')

query_id = wr.athena.start_query_execution('DROP TABLE IF EXISTS coe_analytics_tmp.ao_cross3_vehjoin7;', database='coe_analytics_tmp')
res = wr.athena.wait_query(query_execution_id=query_id)

query_id = wr.athena.start_query_execution(q, database='coe_analytics_tmp', )
res = wr.athena.wait_query(query_execution_id=query_id)

In [6]:
q = """
CREATE TABLE coe_analytics_tmp.ao_cross3_vehjoin8
WITH (
  format='PARQUET',
  external_location='s3://rimac-analytics-temporal/individuals/Antonio Ordonez/data/base_cross/ao_cross3_vehjoin8'
) AS
select A.*, b.placa, b.veh_marca, b.veh_modelo,  b.veh_clase, b.veh_valor_act, b.veh_valor_nuevo, 
b.veh_anhofab, b.veh_tipo_alto_riesgo, b.veh_nro_asientos, b.veh_color, b.veh_origen
FROM ads_mt.mt_vehiculos B
inner JOIN coe_analytics_tmp.ao_cross3_vehjoin7 A
ON A.cuc = B.veh_cuc and A.periodo_target = B.periodo;
"""

#wr.s3.delete_objects('s3://rimac-analytics-temporal/individuals/Antonio Ordonez/data/base_cross/ao_cross3_vehjoin8')

#query_id = wr.athena.start_query_execution('DROP TABLE IF EXISTS coe_analytics_tmp.ao_cross3_vehjoin8;', database='coe_analytics_tmp')
#res = wr.athena.wait_query(query_execution_id=query_id)

#query_id = wr.athena.start_query_execution(q, database='coe_analytics_tmp', )
#res = wr.athena.wait_query(query_execution_id=query_id)

In [ ]:
q = f"""
CREATE TABLE coe_analytics_tmp.ao_cross3_tmp00
	with (
	 format = 'parquet',
	 parquet_compression = 'snappy',
	 external_location = 's3://rimac-analytics-temporal/individuals/Antonio Ordonez/data/base_cross/ao_cross3_tmp00'
	)  as 
	   with tmp_canales as (
			select canal, id_canalnt
			from bdrrvv_app_datman.per_clientes_detalle
			group by canal, id_canalnt
		)
	  
	  select periodo, cuc , id_producto, riesgo , canal, 
	       cast( ( case when  REPLACE(prima_cont,'[^0-9.]', '') = '' then null
	                    else  REPLACE(prima_cont,'[^0-9.]', '') end )
	                as double) prima_cont
	  from bdrrvv_app_datman.per_clientes_detalle
	  where periodo >= '201501' and periodo <= '201712' 
	  --
	  union all
	  --
	  select  a.periodo, a.cuc, a.id_producto, a.riesgo,
		        a.canaldes canal, 
		        cast(a.prima_emitida as double) prima_cont
	  from mis_clientes.dim_modelo_clientes a
	  where periodo >= '201801'  and periodo <= '{period}' -- limitar perido cerrado
	    and id_producto not in ('4001','4075','9030','1304','1355','4062','1357','4020','4061','4028','1996','3101','4063','1403')
	    and vigencia ='VIGENTE' 
	    and tipo_stock != 'Caduco';
"""

wr.s3.delete_objects('s3://rimac-analytics-temporal/individuals/Antonio Ordonez/data/base_cross/ao_cross3_tmp00')

query_id = wr.athena.start_query_execution('DROP TABLE coe_analytics_tmp.ao_cross3_tmp00;', database='coe_analytics_tmp')
res = wr.athena.wait_query(query_execution_id=query_id)

query_id = wr.athena.start_query_execution(q, database='coe_analytics_tmp', )
res = wr.athena.wait_query(query_execution_id=query_id)

In [33]:
q = """
CREATE TABLE coe_analytics_tmp.ao_cross3_vehjoin8
WITH (
  format='PARQUET',
  external_location='s3://rimac-analytics-temporal/individuals/Antonio Ordonez/data/base_cross/ao_cross3_vehjoin8'
) AS

with t0 as (
    select date_diff('day', date_parse(fecinipol, '%d-%M-%y'), date_parse(fecanul, '%d-%M-%y')) as date_difpol, date_diff('day', date_parse(fecinicert, '%d-%M-%y'), date_parse(fecanucert, '%d-%M-%y')) as date_difcert, date_format(date_parse(D.fecinicert, '%d-%M-%y'), '%Y%m') as periodo, D.idepol, D.numpol, D.p_auto as placa, C.cuc
        from bdrrvv_app_datman.dwr_dm_poliza_veh D
        join bdrrvv_app_datman.per_persona_cucs C on D.id_asegur = C.id_persona
),
t01 as (
    select E.cuc, E.periodo, E.placa, b.veh_marca, b.veh_modelo, b.veh_clase, b.veh_valor_act, b.veh_valor_nuevo, 
    b.veh_anhofab, b.veh_tipo_alto_riesgo, b.veh_nro_asientos, b.veh_color, b.veh_origen
        from t0 E
        left join ads_mt.mt_vehiculos B on E.placa = B.placa and E.periodo = B.periodo
        where (E.date_difpol > 5 or E.date_difpol is null) and (E.date_difcert > 5 or E.date_difcert is null)
),
t1 as (
    select A.*, b.placa, b.veh_marca, b.veh_modelo, b.veh_clase, b.veh_valor_act, b.veh_valor_nuevo, 
    b.veh_anhofab, b.veh_tipo_alto_riesgo, b.veh_nro_asientos, b.veh_color, b.veh_origen, 1 as jerarquia
        from t01 B
        join coe_analytics_tmp.ao_cross3_vehjoin7 A ON A.cuc = B.cuc and A.periodo_target = B.periodo
),
t2 as (
    select A.*, b.placa, b.veh_marca, b.veh_modelo,  b.veh_clase, b.veh_valor_act, b.veh_valor_nuevo, 
    b.veh_anhofab, b.veh_tipo_alto_riesgo, b.veh_nro_asientos, b.veh_color, b.veh_origen, 0 as jerarquia
    FROM ads_mt.mt_vehiculos B
    inner JOIN coe_analytics_tmp.ao_cross3_vehjoin7 A
    ON A.cuc = B.veh_cuc and A.periodo_target = B.periodo
)
select *
from t1 
union all 
select *
from t2;
"""

wr.s3.delete_objects('s3://rimac-analytics-temporal/individuals/Antonio Ordonez/data/base_cross/ao_cross3_vehjoin8')

query_id = wr.athena.start_query_execution('DROP TABLE IF EXISTS coe_analytics_tmp.ao_cross3_vehjoin8;', database='coe_analytics_tmp')
res = wr.athena.wait_query(query_execution_id=query_id)

query_id = wr.athena.start_query_execution(q, database='coe_analytics_tmp', )
res = wr.athena.wait_query(query_execution_id=query_id)

In [32]:
q = """
CREATE TABLE coe_analytics_tmp.ao_tmp
WITH (
  format='PARQUET',
  external_location='s3://rimac-analytics-temporal/individuals/Antonio Ordonez/data/base_cross/ao_tmp'
) AS

with t0 as (
    select date_diff('day', date_parse(fecinipol, '%d-%M-%y'), date_parse(fecanul, '%d-%M-%y')) as date_difpol, date_diff('day', date_parse(fecinicert, '%d-%M-%y'), date_parse(fecanucert, '%d-%M-%y')) as date_difcert, date_format(date_parse(D.fecinicert, '%d-%M-%y'), '%Y%m') as periodo, D.idepol, D.numpol, D.p_auto as placa, C.cuc
        from bdrrvv_app_datman.dwr_dm_poliza_veh D
        join bdrrvv_app_datman.per_persona_cucs C on D.id_asegur = C.id_persona
)
-- t01 as (
    select E.cuc, E.periodo, E.placa, b.veh_marca, b.veh_modelo, b.veh_clase, b.veh_valor_act, b.veh_valor_nuevo, 
    b.veh_anhofab, b.veh_tipo_alto_riesgo, b.veh_nro_asientos, b.veh_color, b.veh_origen
        from t0 E
        left join ads_mt.mt_vehiculos B on E.placa = B.placa and E.periodo = B.periodo
        where (E.date_difpol > 5 or E.date_difpol is null) and (E.date_difcert > 5 or E.date_difcert is null);

"""

wr.s3.delete_objects('s3://rimac-analytics-temporal/individuals/Antonio Ordonez/data/base_cross/ao_tmp')

query_id = wr.athena.start_query_execution('DROP TABLE IF EXISTS coe_analytics_tmp.ao_tmp;', database='coe_analytics_tmp')
res = wr.athena.wait_query(query_execution_id=query_id)

query_id = wr.athena.start_query_execution(q, database='coe_analytics_tmp', )
res = wr.athena.wait_query(query_execution_id=query_id)